In [1]:
import numpy as np 
from scipy.io import loadmat 

In [2]:
#Load data
xtr = loadmat('data/Xtr_spam.mat')['Xtr_spam'].T
xte = loadmat('data/Xte_spam.mat')['Xte_spam'].T

ytr = loadmat('data/ytr_spam.mat')['ytr_spam'].flatten()
yte = loadmat('data/yte_spam.mat')['yte_spam'].flatten()

ytr[ytr == -1] = 0
yte[yte == -1] = 0

FileNotFoundError: [Errno 2] No such file or directory: 'data/Xtr_spam.mat'

In [ ]:
sigma1 = xtr[np.where(ytr==1)].std(0)
sigma2 = xtr[np.where(ytr==0)].std(0)
sigma = (sigma1+sigma2)/2

S = np.eye(xtr.shape[1])*sigma
print(S.shape)

(57, 57)


2.12
Assuming classes are normally distributed


In [ ]:
#Estimate means 
mu1 = np.mean(xtr[ytr == 0], axis=0)
mu2 = np.mean(xtr[ytr == 1], axis=0)
print(mu1.shape, mu2.shape)

(57,) (57,)


In [ ]:
def normaldist(x, mu, sigma):
    exponent = -0.5 * (x - mu) @ np.linalg.inv(sigma) @ (x - mu).T
    denominator = (2 * np.pi) ** (len(mu) / 2) * np.sqrt(np.linalg.det(sigma))
    return np.diag(np.exp(exponent)/denominator)

In [ ]:
p1, p2 = normaldist(xtr, mu1, S), normaldist(xtr, mu2, S)
probs = np.array([p1, p2])
pred = np.argmax(probs, axis=0)

print(f'Accuracy on spam train data is: {np.round(np.sum(pred == ytr)/len(xtr))}')

p1, p2 = normaldist(xte, mu1, S), normaldist(xte, mu2, S)
probs = np.array([p1, p2])
pred = np.argmax(probs, axis=0)

print(f'Accuracy on spam test data is: {np.round(np.sum(pred == yte)/len(xte))}')

Accuracy on spam train data is: 1.0
Accuracy on spam test data is: 1.0


Doing 2.35 now

In [ ]:
def classifyNN(traindata, trainlabels, testdata, NN=1):
    '''
    Input:
        traindata: Data to be used to assign a classification to testdata
        trainlabels: Labels for each traindata point
        testdata: Data which we wish to classify
        NN: Number of nearest neighbours. By default equals to 1
    This function could be switched out with a scipy function which does the same, 
    but I find value in trying to do it yourself before using premade versions
    
    For every datapoint in testdata, we need to calculate the distance between it and all traindata points,
    then find the NN closest points and classify it to the majority class
    '''
    
    testpredictions = []
    
    for x in testdata:
        distances = np.linalg.norm(traindata - x, axis=-1)
        sortdistances = np.argsort(distances)
        NNclosestpoints = sortdistances[:NN]
        closestlabels = trainlabels[NNclosestpoints]
        mostcommonpoint = np.argmax(np.bincount(closestlabels))
        testpredictions.append(mostcommonpoint)
    
    return np.asarray(testpredictions)


In [ ]:
pred_xte = classifyNN(xtr, ytr, xte, NN=1)
print(f'Accuracy on spam test data when NN=1 is: {np.round(np.sum(pred_xte == yte)/len(xte))}')

pred_xte = classifyNN(xtr, ytr, xte, NN=3)
print(f'Accuracy on spam test data when NN=3 is: {np.round(np.sum(pred_xte == yte)/len(xte))}')

Accuracy on spam test data when NN=1 is: 1.0
Accuracy on spam test data when NN=3 is: 1.0
